In [3]:
import cv2
import pandas as pd
import numpy as np
from datetime import datetime
import threading
import time
import os
import tkinter as tk
from tkinter import ttk, messagebox

# Load the trained LBPH model and label mapping
model_filename = "trained_model.yml"
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read(model_filename)

label_map_df = pd.read_csv("label_map.csv")
inv_label_map = {row['Label']: row['UserID'] for index, row in label_map_df.iterrows()}

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
if face_cascade.empty():
    raise IOError("Cannot load cascade file. Check the file path.")

# Global variables for the recognition thread
stop_event = threading.Event()
attendance_df = pd.DataFrame(columns=["Name", "Date", "Time"])
attendance_recorded = set()  # To avoid duplicate logging

# Recognition function running in a separate thread
def run_recognition(auto_shutdown_delay=60):
    cap = cv2.VideoCapture(0)
    start_time = time.time()
    
    while not stop_event.is_set():
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_roi = gray_frame[y:y+h, x:x+w]
            label, confidence = recognizer.predict(face_roi)
            if confidence < 100:
                name = inv_label_map.get(label, "Unknown")
            else:
                name = "Unknown"

            if name != "Unknown" and name not in attendance_recorded:
                current_time = datetime.now()
                new_entry = {
                    "Name": name,
                    "Date": current_time.strftime("%Y-%m-%d"),
                    "Time": current_time.strftime("%H:%M:%S")
                }
                global attendance_df
                attendance_df = pd.concat([attendance_df, pd.DataFrame([new_entry])], ignore_index=True)
                attendance_recorded.add(name)
                print("Attendance logged:", new_entry)
        
        # Auto shutdown after delay
        if time.time() - start_time > auto_shutdown_delay:
            print("Auto shutdown delay reached. Stopping recognition...")
            break

        time.sleep(0.1)

    cap.release()
    print("Recognition thread finished.")

# Tkinter GUI
def start_recognition():
    # Clear any previous stop event and start a new recognition thread
    stop_event.clear()
    thread = threading.Thread(target=run_recognition, args=(60,), daemon=True)
    thread.start()
    messagebox.showinfo("Info", "Recognition started.")

def stop_recognition():
    stop_event.set()
    messagebox.showinfo("Info", "Recognition stopped.")

def refresh_attendance():
    # Clear the treeview and populate with latest attendance data
    for row in attendance_tree.get_children():
        attendance_tree.delete(row)
    for index, row in attendance_df.iterrows():
        attendance_tree.insert("", "end", values=(row["Name"], row["Date"], row["Time"]))

def save_attendance():
    filename = "attendance_recognition_gui.xlsx"
    attendance_df.to_excel(filename, index=False)
    messagebox.showinfo("Info", f"Attendance saved to {filename}")

# Set up the main window
root = tk.Tk()
root.title("Face Recognition Attendance System")
root.geometry("600x400")

# Buttons to control recognition
btn_frame = tk.Frame(root)
btn_frame.pack(pady=10)

start_btn = tk.Button(btn_frame, text="Start Recognition", command=start_recognition)
start_btn.pack(side="left", padx=5)

stop_btn = tk.Button(btn_frame, text="Stop Recognition", command=stop_recognition)
stop_btn.pack(side="left", padx=5)

refresh_btn = tk.Button(btn_frame, text="Refresh Attendance", command=refresh_attendance)
refresh_btn.pack(side="left", padx=5)

save_btn = tk.Button(btn_frame, text="Save Attendance", command=save_attendance)
save_btn.pack(side="left", padx=5)

# Attendance table using ttk Treeview
attendance_tree = ttk.Treeview(root, columns=("Name", "Date", "Time"), show="headings", height=10)
attendance_tree.heading("Name", text="Name")
attendance_tree.heading("Date", text="Date")
attendance_tree.heading("Time", text="Time")
attendance_tree.pack(pady=10, fill="both", expand=True)

# Start the Tkinter main loop
root.mainloop()


Attendance logged: {'Name': 'Atharv', 'Date': '2025-02-23', 'Time': '01:03:51'}
Recognition thread finished.


In [10]:
import sqlite3
from datetime import datetime

def create_database(db_name="attendance.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT UNIQUE NOT NULL
    )
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Attendance (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        date TEXT,
        time TEXT,
        FOREIGN KEY(user_id) REFERENCES Users(user_id)
    )
    """)
    conn.commit()
    conn.close()
    print("Database and tables created successfully.")

def add_user_to_db(name, db_name="attendance.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Users (name) VALUES (?)", (name,))
        conn.commit()
        user_id = cursor.lastrowid
        print(f"User '{name}' added with ID: {user_id}")
    except sqlite3.IntegrityError:
        cursor.execute("SELECT user_id FROM Users WHERE name = ?", (name,))
        user_id = cursor.fetchone()[0]
        print(f"User '{name}' already exists with ID: {user_id}")
    conn.close()
    return user_id

def log_attendance(user_id, db_name="attendance.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    current_date = datetime.now().strftime("%Y-%m-%d")
    current_time = datetime.now().strftime("%H:%M:%S")
    cursor.execute("INSERT INTO Attendance (user_id, date, time) VALUES (?, ?, ?)",
                   (user_id, current_date, current_time))
    conn.commit()
    conn.close()
    print(f"Attendance logged for user ID {user_id} at {current_date} {current_time}")

# Create the database and tables (run this once at the start)
create_database()


Database and tables created successfully.
